In [1]:
#Authors: Adam Lewandowski, Ivan Sladkov, Patrick English
import numpy as np
import tensorflow as tf
import pandas as pd
import string
import nltk

#from spellchecker import SpellChecker
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

In [4]:
raw_tweets_df = pd.read_csv("data/training.1600000.processed.noemoticon.csv", encoding = "ISO-8859-1", header=None)
raw_tweets_df

FileNotFoundError: [Errno 2] No such file or directory: 'data/training.1600000.processed.noemoticon.csv'

In [3]:
#Baseline processing
tknzr = TweetTokenizer()

#This can take a while
#tokenized_tweets = raw_tweets_df[5].copy().apply(lambda x : x.lower())

# If you don't want to wait
tokenized_tweets = raw_tweets_df[5][:1600].copy().apply(lambda x : x.lower().encode('ascii', 'ignore'))

tokenized_tweets = tokenized_tweets.apply(lambda x : tknzr.tokenize(x))

In [4]:
#Should things like ... stay? 
def remove_punctuation(tweets):
    t = tweets.copy()
    def remove_punct(tokens):
        if '.' in tokens:
            tokens.remove('.')
        if '?' in tokens:
            tokens.remove('?')
        if '!' in tokens:
            tokens.remove('!')
        if ',' in tokens:
            tokens.remove(',')
        if '\'' in tokens:
            tokens.remove('\'')
        if '"' in tokens:
            tokens.remove('"')
        if '#' in tokens:
            tokens.remove('#')
        if '' in tokens:
            tokens.remove('#')
            
        return tokens
    return t.apply(lambda tweet : remove_punct(tweet))

def remove_stop_words(tweets):
    t = tweets.copy()
    from nltk.corpus import stopwords
    sw = set(stopwords.words('english')) 
    return t.apply(lambda tweet:[w for w in tweet if not w in sw])

#TODO add spell correction
def correct_spelling(tweets):
    return tweets

def pos_tag(tweets):
    t = tweets.copy()
    return t.apply(lambda tweet: nltk.pos_tag(tweet))

#TBD
def emoticon_transcoder(tweets):
    return tweets

def remove_pos_tags(tweets):
    t = tweets.copy()
    return t.apply(lambda tweet: [w[0] for w in tweet])

def identity_tokenizer(text):
    return text

def scikit_vectorize(tweets, min_freq):
    t = tweets.copy()
    vectorizer = TfidfVectorizer(lowercase=False, tokenizer=identity_tokenizer, min_df=min_freq, dtype=np.float32)
    vectors = vectorizer.fit_transform(t)
    feature_names = vectorizer.get_feature_names()
    dense = vectors.todense()
    denselist = dense.tolist()
    return pd.DataFrame(denselist, columns=feature_names)

def str_list_to_list(tweets):
    t = tweets.copy()
    import ast 
    return t.apply(lambda x: ast.literal_eval(x))

def lemmatize(tweets):
    t = tweets.copy()
    
    def get_wordnet_pos(treebank_tag):
        from nltk.corpus import wordnet
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN
    lem = WordNetLemmatizer()
    def lemmatize_word(w):
        transcoded_tag = get_wordnet_pos(w[1])
        wl = lem.lemmatize(w[0], transcoded_tag)
        return (wl,w[1])
    return t.apply(lambda tweet:[lemmatize_word(w) for w in tweet])

def remove_punct(tokens):
        if '.' in tokens:
            tokens.remove('.')
        if '?' in tokens:
            tokens.remove('?')
        if '!' in tokens:
            tokens.remove('!')
        if ',' in tokens:
            tokens.remove(',')
        return tokens
    
def remove_one_char_trash(tweets):
    return tweets.apply(lambda tweet: [s for s in tweet if len(s) >= 2])

arr = ["gdfgfd", "]","rer"]
arr = [s for s in arr if len(s) >= 2]
print(arr)

['gdfgfd', 'rer']


In [5]:
#Pre-processing
#comment lines to skip some steps
#You need to run things in this order
tweets = remove_punctuation(tokenized_tweets)
tweets = remove_one_char_trash(tweets)
tweets = remove_stop_words(tweets)
tweets = pos_tag(tweets)
tweets = lemmatize(tweets)
tweets = remove_pos_tags(tweets)
#tweets = merge_tokens(tweets)

In [7]:

import os
if os.path.exists("processed_tweets.csv"):
    os.remove("processed_tweets.csv")

tweets.to_csv("processed_tweets.csv", encoding = "ISO-8859-1", header=None, index=False)


In [8]:
tweets = pd.read_csv("processed_tweets.csv", encoding = "ISO-8859-1", header=None)[0].copy()
tweets = str_list_to_list(tweets)

In [9]:
#print(tweets)
tweetsVector = scikit_vectorize(tweets, 2)

#print(tweets.array)
#tweetsVector
#print(tweetsVector)

In [11]:
if os.path.exists("vec.csv"):
    os.remove("vec.csv")

tweetsVector.to_csv("vec.csv", encoding = "ISO-8859-1", index=False)

In [1]:
print(len(tweetsVector.iloc[0]))
#print(tweetsVector.keys())

NameError: name 'tweetsVector' is not defined